In [1]:
import umap
import hdbscan
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import urllib.request
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
df = pd.read_csv("./history.csv")
urls = df['url']

In [5]:
docs = []
for url in urls:
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        html = urlopen(req).read()
        soup = BeautifulSoup(html,'html5lib')
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        docs.append(text)
    except Exception as e:
        print(e)

Remote end closed connection without response
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
<urlopen error [WinError 2] The system cannot find the file specified: "C:\\Users\\balsh\\Downloads\\Balsher's Resume.pdf">
<urlopen error [WinError 2] The system cannot find the file specified: "C:\\Users\\balsh\\Downloads\\Balsher's Resume.pdf">
HTTP Error 401: Unauthorized
HTTP Error 401: Unauthorized
Remote end closed connection without response
Remote end closed connection without response
<urlopen error [WinError 3] The system cannot find the path specified: 'C:\\Users\\balsh\\Desktop\\MISC\\New folder\\Balsher_resume.pdf'>
<urlopen error [WinError 2] The system cannot find the file specified: "C:\\Users\\balsh\\Desktop\\MISC\\Balsher's Resume.pdf">
<urlopen error [WinError 3] The system cannot find the p

In [4]:
# Load sentence transformer model
sentence_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Define UMAP model to reduce embeddings dimension
umap_model = umap.UMAP(n_neighbors=15,
                       n_components=10,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False,
                       random_state = 42)

# Define HDBSCAN model to perform documents clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

# Create BERTopic model
topic_model = BERTopic(top_n_words=10,
                       n_gram_range=(1,2),
                       calculate_probabilities=True,
                       umap_model= umap_model,
                       hdbscan_model=hdbscan_model,
                       embedding_model=sentence_model,
                       verbose=True)

In [6]:
# Train model, extract topics and probabilities
topics, probabilities = topic_model.fit_transform(docs)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

2021-12-29 13:13:57,173 - BERTopic - Transformed documents to Embeddings
E:\LG\venv\lib\site-packages\numba\np\ufunc\parallel.py:366: NumbaWarning: The TBB threading layer requires TBB version 2021 update 1 or later i.e., TBB_INTERFACE_VERSION >= 12010. Found TBB_INTERFACE_VERSION = 11103. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-12-29 13:14:06,564 - BERTopic - Reduced dimensionality with UMAP
2021-12-29 13:14:06,667 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
print(topic_model.get_topics())

topic_model.visualize_hierarchy()